SQL Case Study - Country Club

In [29]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("sqlite_db_pythonsqlite.db")

def q(sql):
    return pd.read_sql_query(sql, conn)

Question 1:
Find the names of all facilities that charge a fee to members.

In [30]:
q("""
SELECT name
FROM Facilities
WHERE membercost > 0;
""")

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


Question 2:
How many facilities do not charge a fee to members.

In [31]:
q("""
SELECT COUNT(*) AS count_no_fee
FROM Facilities
WHERE membercost = 0;
""")

,count_no_fee
0,4


Question 3:
Find the facilities where the member cost is less than twenty percent of the monthly maintenance.

In [32]:
q("""
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0
  AND membercost < 0.20 * monthlymaintenance;
""")

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


Question 4:
Retrieve the details of facilities with ID 1 and 5.

In [33]:
q("""
SELECT *
FROM Facilities
WHERE facid IN (1, 5);
""")

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Question 5:
Label facilities as cheap or expensive based on monthly maintenance greater than 100.

In [34]:
q("""
SELECT
  name,
  monthlymaintenance,
  CASE
    WHEN monthlymaintenance > 100 THEN 'expensive'
    ELSE 'cheap'
  END AS cost_label
FROM Facilities;
""")

,name,monthlymaintenance,cost_label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Question 6:
Find the first and last name of the last member who signed up.

In [35]:
q("""
SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);
""")

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


Question 7:
List all members who have used a tennis court.

In [36]:
q("""
SELECT DISTINCT
  f.name AS court_name,
  m.firstname || ' ' || m.surname AS member_name
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY member_name;
""")

,court_name,member_name
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 1,David Farrell
8,Tennis Court 2,David Farrell
9,Tennis Court 2,David Jones


Question 8:
Find bookings on 2012 09 14 that cost more than 30.

In [37]:
q("""
SELECT
  f.name AS facility_name,
  m.firstname || ' ' || m.surname AS member_name,
  (CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END) * b.slots AS cost
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE date(b.starttime) = '2012-09-14'
  AND ((CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END) * b.slots) > 30
ORDER BY cost DESC;
""")

,facility_name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Question 9:
Same as Q8 but using a subquery.

In [38]:
q("""
SELECT facility_name, member_name, cost
FROM (
  SELECT
    f.name AS facility_name,
    m.firstname || ' ' || m.surname AS member_name,
    (CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END) * b.slots AS cost
  FROM Bookings b
  JOIN Facilities f ON b.facid = f.facid
  JOIN Members m ON b.memid = m.memid
  WHERE date(b.starttime) = '2012-09-14'
) x
WHERE cost > 30
ORDER BY cost DESC;
""")

,facility_name,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Question 10:
List facilities with total revenue less than 1000.

In [39]:
q("""
SELECT
  f.name AS facility_name,
  SUM((CASE WHEN b.memid = 0 THEN f.guestcost ELSE f.membercost END) * b.slots) AS revenue
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
GROUP BY f.facid, f.name
HAVING revenue < 1000
ORDER BY revenue;
""")

,facility_name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270
